In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv


In [3]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
creds = '{"username":"ringoospina","key":"7f4f8a367b147077f626415a60bea2c3"}' #insert your own from kaggle api


In [4]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser() 

if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [5]:
path = Path('us-patent-phrase-to-phrase-matching')


if not iskaggle and not path.exists(): 
    import zipfile,kaggle # import the libraries

    kaggle.api.competition_download_cli(str(path)) 
    zipfile.ZipFile(f'{path}.zip').extractall(path)

### Importing an EDA (Exploratory Data Analysis library)

In [6]:
if iskaggle:
    path = Path('../input/us-patent-phrase-to-phrase-matching') 
    ! pip install -q datasets

In [7]:
!ls {path} # list the files in the directory

sample_submission.csv  test.csv  train.csv


In [8]:
import pandas as pd

In [9]:
# reading the train.csv file using pandas
df = pd.read_csv(path/"train.csv")
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [10]:
# Getting some interesting specs about the data
df.describe() # general specs of the data

,score
count,36473.000000
mean,0.362062
std,0.258335
min,0.000000
25%,0.250000
50%,0.250000
75%,0.500000
max,1.000000


In [11]:
df.describe(include='object') # column specs

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


# Concatinating the relevant data together in one column

In [12]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor
df

#df.input.head()


,id,anchor,target,context,score,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1: A47; TEXT2: abatement of pollution; ANC...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1: A47; TEXT2: forest region; ANC1: abatement
...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,TEXT1: B44; TEXT2: wooden article; ANC1: wood ...
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,TEXT1: B44; TEXT2: wooden box; ANC1: wood article
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,TEXT1: B44; TEXT2: wooden handle; ANC1: wood a...
36471,756ec035e694722b,wood article,wooden material,B44,0.75,TEXT1: B44; TEXT2: wooden material; ANC1: wood...


In [13]:
df.input.head() # the new added column

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

# Tokenization
In this section we will perform the 

-Toknization (splitting each texts into tokens) 

-Numericalization: Converting each token into a number

In [14]:
# because transformers use a Dataset for storing a dataset after doing the tokenization

from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(df)

In [15]:
#picking an already implemented model
model_nm = 'microsoft/deberta-v3-small'

In [16]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer #import the model and tokenizer classes

tokz = AutoTokenizer.from_pretrained(model_nm) # load the tokenizer



tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [17]:
#example how the tokenizer works
tokz.tokenize("Whatzaaaap my bros of Kaggle, I'm Xhon Taraj")

['▁What',
 'z',
 'aaaa',
 'p',
 '▁my',
 '▁bros',
 '▁of',
 '▁K',
 'aggle',
 ',',
 '▁I',
 "'",
 'm',
 '▁X',
 'hon',
 '▁Tara',
 'j']

In [18]:
# tokenizing the column "input" of our dataset ds

def tok_func(x):
    return tokz(x["input"]) #column input to be tokenized (adds some columns in our dataset that do the he "numericalization" of column "input")

In [19]:
# tokenizing every row now fast in batches and in paralel

tok_ds = ds.map(tok_func, batched=True) # batch size by default is 1000

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [20]:
tok_ds # now new columns are added, important for us are the input_ids which gives us the "numericalization" of column "input"


Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

In [21]:
tok_ds[0]

{'id': '37d61fd2272659b1',
 'anchor': 'abatement',
 'target': 'abatement of pollution',
 'context': 'A47',
 'score': 0.5,
 'input': 'TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 'input_ids': [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
row =tok_ds[0]

row['input'],row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [23]:
#example of the number fo the word "of" in the "TEXT2: abatement ->of<- pollution"
tokz.vocab['▁of']

265

In [24]:
# we rename our column "score" into "labels" because transformers work with this keyword.

tok_ds =tok_ds.rename_columns({'score':'labels'})

In [55]:
# reading the test set which we will be using in the end after training the model
# we call it eval to not confuse it with the test:dataset we are going to create below 
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor 
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [32]:
# dividing our data into validation and train set

dds = tok_ds.train_test_split(0.25, seed=42) #25% of data will be validation ,75% train
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

## Training the model

In [34]:
from transformers import TrainingArguments,Trainer 


In [40]:
bs =128 # batch size
epochs = 4
lr = 8e-5 # learning rate

In [41]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [48]:
# creating a correlation function
def corr(x,y): 
    return np.corrcoef(x,y)[0][1] 


def corr_d(eval_pred): 
    return {'pearson': corr(*eval_pred)} # returns the correlation as a dictionary with the key(label) 'pearson'


In [49]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)


trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'], 
                  tokenizer=tokz, compute_metrics=corr_d) 

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [50]:
trainer.train() #training the model

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.025167,0.799840
2,No log,0.022671,0.821986
3,0.029900,0.022271,0.832017
4,0.029900,0.022520,0.833633


TrainOutput(global_step=856, training_loss=0.02317588852944775, metrics={'train_runtime': 264.0387, 'train_samples_per_second': 414.394, 'train_steps_per_second': 3.242, 'total_flos': 715555561923540.0, 'train_loss': 0.02317588852944775, 'epoch': 4.0})

In [57]:
# prediction from the test (eval) test
preds = trainer.predict(eval_ds).predictions.astype(float)

preds

array([[ 0.52783203],
       [ 0.69140625],
       [ 0.4440918 ],
       [ 0.33984375],
       [-0.0317688 ],
       [ 0.49926758],
       [ 0.52197266],
       [-0.01318359],
       [ 0.2286377 ],
       [ 1.11230469],
       [ 0.26782227],
       [ 0.26196289],
       [ 0.76904297],
       [ 0.87451172],
       [ 0.77978516],
       [ 0.4453125 ],
       [ 0.2668457 ],
       [-0.03967285],
       [ 0.64208984],
       [ 0.36401367],
       [ 0.46533203],
       [ 0.23132324],
       [ 0.13696289],
       [ 0.23547363],
       [ 0.57617188],
       [-0.02688599],
       [-0.04473877],
       [-0.00993347],
       [-0.04196167],
       [ 0.65039062],
       [ 0.33569336],
       [-0.01271057],
       [ 0.77001953],
       [ 0.50195312],
       [ 0.390625  ],
       [ 0.2019043 ]])

In [58]:
# because some predictions are <0 or >1 , we make the one that are <0 -> 0 and the ones >1 -> 1

preds = np.clip(preds, 0, 1)

preds

array([[0.52783203],
       [0.69140625],
       [0.4440918 ],
       [0.33984375],
       [0.        ],
       [0.49926758],
       [0.52197266],
       [0.        ],
       [0.2286377 ],
       [1.        ],
       [0.26782227],
       [0.26196289],
       [0.76904297],
       [0.87451172],
       [0.77978516],
       [0.4453125 ],
       [0.2668457 ],
       [0.        ],
       [0.64208984],
       [0.36401367],
       [0.46533203],
       [0.23132324],
       [0.13696289],
       [0.23547363],
       [0.57617188],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.65039062],
       [0.33569336],
       [0.        ],
       [0.77001953],
       [0.50195312],
       [0.390625  ],
       [0.2019043 ]])

In [59]:
# submit csv file creation 
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=True)


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1109